In [ ]:
!pip install -U scikit-fuzzy
import skfuzzy.cluster as fuzz
import numpy as np
import pandas as pd

# Importar o conjunto de dados
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.data'
df = pd.read_csv(url, skiprows=3)
df1 = df.iloc[:,4:9] #PRIMEIRA DIVISAO DO DATASET: 4 A 9

# Converter os dados para uma matriz NumPy
X = df1.iloc[:, :-1].to_numpy(dtype=np.float32)
# Separar as classes dos dados e normalizá-los
y = df1.iloc[:, -1].to_numpy()
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Executar o algoritmo FCM com a distância de City-Block e a partição fuzzy em 7 grupos, repetindo-o 50 vezes para obter o melhor resultado
n_clusters = 7
best_obj_func = np.inf
best_U = None # Melhor resultado

# Executando o algoritmo FCM com a distância city-block personalizada
for i in range(50):
    cntr, u, u0, d, jm, p, fpc = fuzz.cmeans(data=X.T, c=n_clusters, m=2, error=1e-3, maxiter=1000, metric='cityblock')
    if jm.mean() < best_obj_func:
        best_obj_func = jm.mean()
        best_U = u

# Imprimindo os rótulos dos clusters e os centros
labels = np.argmax(u, axis=0)
centers = cntr.T
print(labels)
print('------------------------')
print(centers)
print('------------------------')
print(best_U)

#Melhor resultado de acordo com a função objetivo
best_jm = np.inf

cntr, u, _, _, jm, _, _ = fuzz.cmeans(data=X.T, c=n_clusters, m=2, error=1e-3, maxiter=1000, metric='cityblock')
if jm.mean() < best_jm:
    best_jm = jm.mean()
    best_U = u
print("Melhor resultado de acordo com a Função objetivo:", best_jm)

#Modiffied Partition Coefficient e Partition entropy:
pc = np.sum(np.max(best_U, axis=0)) / X.shape[0]
print("Modified Partition Coefficient:", pc)

pe = - np.sum(best_U * np.log(best_U)) / X.shape[0]
print("Partition Entropy:", pe)

# Fuzzy C-Means para obter a partição CRISP
cntr, u, u0, d, jm, p, fpc = fuzz.cmeans(data=X.T, c=n_clusters, m=2, error=1e-3, maxiter=1000, metric='cityblock')
labels = np.argmax(u, axis=0)

# Índice Rand
from sklearn.metrics import adjusted_rand_score
true_labels = np.array([int(i)-1 for i in y]) # Converter os labels em y
ri = adjusted_rand_score(true_labels, labels)
print("Índice Rand:", ri)

#F-measure adaptada para agrupamento
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(true_labels, labels)

# Calcular precisao, recall, e F-measure para cada cluster
precision = []
recall = []
f_measure = []
for i in range(n_clusters):
    tp = cm[i, i]
    fp = cm[:, i].sum() - tp
    fn = cm[i, :].sum() - tp
    if tp == 0:
        precision.append(0)
        recall.append(0)
        f_measure.append(0)
    else:
        precision.append(tp / (tp + fp))
        recall.append(tp / (tp + fn))
        f_measure.append(2 * precision[-1] * recall[-1] / (precision[-1] + recall[-1]))

# Calcular o f-measure adaptada para agrupamento
f_measure_group = sum(f_measure) / n_clusters
print("F-measure adaptada para agrupamento:", f_measure_group)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894086 sha256=60152c5dc78c30b6767998b809bf3ddd009daf813d27b1f9b5a85a25407d7b12
  Stored in directory: /root/.cache/pip/wheels/32/2c/a1/a90a7d7dd8448ec029f298a61f3490275e99b17aa348be675c
Successfully built scikit-fuzzy
[1 1 1 1 1 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1
 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 3 1 0 6 2 1 5 1
 1 1 1 5 1 4 1 0 1 4 1 1 0 1 1 1 1 1 0 0 0 0 0 1 4 0 4 0 1 1 0 1 0 1 0 0 0
 1 1 1 4 4 2 4 1 6 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 1 2 6 0 4 2 1 0 0 0 0 4 0 6 0 0 4 0 6 4 0 2 0 4 2 0 0 4 2 0 0 1 0 0 1 1
 0 0 0 0 0 0 0 0 1 4 0 0 0 1 1 0 0 1 1 1 0 1 0 0 1]
------------------------
[[-2.05293214e-01 -2.0969